In [12]:
import numpy as np 
import matplotlib.pyplot as mat
import math
import time
import sympy as sym
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import linprog
from scipy.optimize import minimize
from itertools import combinations
from IPython.display import display

# Se declara la función objetivo como el negativo de la función del problema planteado, pues para estar en la forma
# estándar, se requiere minimizar la función. 
def fobjetivo(x):
    return 26*x[0]-26*x[1]

def inList(array, lista):
    for element in lista:
        if np.array_equal(element, array):
            return True
    return False

def func(A,b):
    
    # Se crea un diccionario cuyas llaves y valores ambas son listas vacías.
    dir = {'Solución':[], 'Base':[]}
    
    # Se determina la dimensión de la matriz y vector de entrada.
    sizeA = np.shape(A) # m ecuaciones
    sizeb = np.shape(b) # n incógnitas
    
    # Se usa la función combinations para encontrar todas las combinaciones posibles que se pueden hacer
    # con las m ecuaciones y n incógnitas. Las componentes de cada tupla que retorna la función indica qué
    # columnas de la matriz se eliminan, es decir, qué variables se están mandando a cero para resolver el 
    # sistema de ecuaciones resultante. La cantidad de combinaciones que resultan es la cantidad de soluciones
    # básicas que tiene el problema de optimización. 
    combs = combinations(list(range(0,sizeA[1])),(sizeA[1]-sizeA[0]))
    
    # Vamos a examinar todas las posibles combinaciones de variables igualadas a cero.
    for item in list(combs):
        
        # Se crea una copia de la matriz, llamada A_1
        A_1 = A.copy()
        
        # Se eliminan las item-ésimas (números de la tupla) columnas (indicadas por axis=1) de A_1, es decir, se mandan
        # a cero las variables correspondientes a esas columnas. 
        A_1 = np.delete(A_1,item,axis=1)
        
        # Se crea otra copia de la matriz, llamada A_2
        A_2 = A.copy()
        
        # Se ponen las item-ésimas (números de la tupla) columnas de A_2 en cero para indicar cómo queda el sistema cuando 
        # se mandan esas variables a cero. Estas son las bases correspondientes a cada solución básica.
        A_2[:,item] = 0
        
        # La solución básica está dada por la solución del sistema resultante al mandar las m-n variables a cero.
        # Si la matriz del sistema de ecuaciones resultante es singular, se agrega la palabra 'Singular' a las llaves, y su
        # respectiva base a los valores.
        if (np.linalg.det(A_1)==0):
            dir['Solución'].append('Matriz singular')
            dir['Base'].append(A_2)
        # Si la matriz sí se puede invertir, se encuentra la solución básica sin problema. Después de encontrar estos valores,
        # se vuelven a agregar las columnas que se eliminaron previamente, y se agregan como un cero en la item-ésima
        # componente de cada solución básica. 
        else:
            ans = np.dot(np.linalg.inv(A_1),b)
            ans = np.round(ans,3)
            # Se usa este ciclo para recorrer las item-ésimas componentes de las soluciones básicas (las que se habían
            # eliminado previamente). En cada componente se agrega un cero.
            for i in item:
                ans = np.insert(ans, i, 0, axis=0)
            # Se agregan las soluciones básicas a la llave 'Solución' del diccionario, y las matrices resultantes al valor 'Base'.
            dir['Solución'].append(ans)
            dir['Base'].append(A_2)
                
                               
    return dir


def fun(A,b):
    
    # Se crea un diccionario con dos llaves. Los valores de la llave 'Factibles' corresponden a las soluciones básicas 
    # factibles. Los valores de la llave 'Base' corresponden a las bases correspondientes a las soluciones básicas 
    # factibles.
    factibles = {'Factibles': [], 'Base': []}
    
    # Se llama a la función creada en el punto 1 para determinar cuáles son las soluciones básicas.
    res1 = func(A,b)
    
    # Las soluciones básicas corresponden a los valores de la llave 'Solución' del diccionario entregado por la función
    # del punto 1. Las bases de estas soluciones básicas corresponden a los valores de la llave 'Base' del diccionario 
    # entregado por la función del punto 1.
    sols_basicas = res1['Solución']
    bases_basicas = res1['Base']

    # Las soluciones básicas factibles corresponden a los valores de la llave 'Solución' del diccionario creado
    # anteriormente. Las bases de estas soluciones básicas factibles corresponden a los valores de la llave 'Base' 
    # del diccionario creado anteriormente. 
    sols_factibles = factibles['Factibles']
    bases_factibles = factibles['Base']
   
    # Se crean tres listas vacías. En la lista 'feval' se guardan los valores de la función objetivo evaluada en las 
    # soluciones básicas. En la lista 'feval_factible' se guardan los valores de la función objetivo evaluada en las
    # soluciones básicas facitbles. En la lista 'tipos' se guardan las distintas clasificaciones que se le pueden dar 
    # a las soluciones del problema de optimización. Estos strings pueden ser 'Mejor', 'Peor', 'Básica factible' y 
    # 'Solución básica'.
    feval = []
    feval_factible = []
    tipos = []
    
    # Se recorren todas las soluciones básicas. Si todas las componentes del vector correspondiente a una solución básica
    # son mayores o iguales a cero, esta solución es básica y por lo tanto se agrega el valor de esta solución básica al
    # diccionario de soluciones básicas factibles. De igual forma, se agrega el valor de la base correspondiente a esa 
    # solución básica al diccionario. 
    for solucion in sols_basicas:
        if all(x>=0 for x in solucion) == True:
            factibles['Factibles'].append(solucion)
            factibles['Base'].append(res1.get('Base', solucion))
        else:
            pass

    # Se recorren todas las soluciones básicas. Se va evaluando el valor de la función objetivo en cada una de las 
    # soluciones básicas. Se agrega este valor obtenido a la lista 'feval'.
    for sol_basica in sols_basicas:
        valor = fobjetivo(sol_basica)
        feval.append(valor)
        
    # Se recorren todas las soluciones básicas factibles. Se va evaluando el valor de la función objetivo en cada una de 
    # las soluciones básicas factibles. Se agrega este valor obtenido a la lista 'feva_factible'.
    for sol_factible in sols_factibles:
        valor_factible = fobjetivo(sol_factible)
        feval_factible.append(valor_factible)
        
    # Se encuentra tanto el valor máximo como el valor mínimo de la función objetivo evaluado en las soluciones básicas
    # factibles.
    maximo = max(feval_factible)
    minimo = min(feval_factible)

    # Se recorren todas las soluciones básicas
    for sol_basica in sols_basicas:
        # Si la solución básica también está en el diccionario de soluciones básicas factibles, sabemos que es una 
        #solución básica factible.
        if inList(sol_basica, sols_factibles):
            # Si al evaluar esta solución básica factible particular obtenemos el valor máximo encontrado previamente, esta 
            # solución corresponde a la peor solución de las soluciones básicas factibles.
            if fobjetivo(sol_basica) == maximo:
                tipos.append("Peor")
            # Si al evaluar esta solución básica factible particular obtenemos el valor mínimo encontrado previamente, esta 
            # solución corresponde a la mejor solución (solución óptima) de las soluciones básicas factibles.
            elif fobjetivo(sol_basica) == minimo:
                tipos.append("Mejor")
            # Si al evaluar esta solución básica factible particular obtenemos un valor diferente al valor máximo o mínimo 
            # encontrado previamente, esta solución corresponde a solamente una solución básica factible.
            else:
                tipos.append("Básica factible")
        # Si la solución básica no está en el diccionario de soluciones básicas factibles, solamente es una solución básica.
        else:
            tipos.append("Solución básica")
            
    # Como se estaba minimizando el valor del negativo de la función objetivo, se procede a multiplicar cada uno de los 
    # valores de todas las soluciones básicas por -1, para obtener el valor que hace sentido en el contexto del problema, 
    # que corresponde a maximizar la función objetivo.
    vals_reales = [x * -1 for x in feval] 

    # Se preseta la información obtenida por la función en una tabla ordenada haciendo uso de la librería pandas.
    tabla1 = pd.DataFrame({
        'Solución': sols_basicas,
        'Base': bases_basicas,
        'Tipo de solución': tipos,
        'Valor': vals_reales
    })

    display(tabla1)
    
    return factibles

In [20]:
# Se declara la matriz de restricciones y su respectivo vector de límites superiores.
A = np.array([[-1,1,-1,0,0,0],[-1,1,0,-1,0,0],[-1,1,0,0,-1,0],[-1,1,0,0,0,-1]])
b = np.array([[2],[1],[7],[4]])

dic_factibles = fun(A,b)
dic_factibles

TypeError: '>=' not supported between instances of 'str' and 'int'

In [26]:
# Se declaran los arreglos correspondientes a la función objetivo (c), la matriz de restricciones (A)
# y el vector de límites superiores b. Se declaran las cotas de cada una de las variables de decisión. 
c = np.array([26, -26])
A = np.array([[1,-1],[1,-1],[1,-1],[1,-1]])
b = np.array([[-2],[-1],[-7],[-4]])
x1_bounds = (0, None)
x2_bounds = (0, None)

start_revised = time.time()
res = linprog(c, A_ub=A, b_ub=b, bounds = np.array([x1_bounds, x2_bounds]), method='simplex', options={"disp": True})
end_revised = time.time()
total_revised = end_revised-start_revised

print(res)
print(f'Tiempo total con el método revised-simplex: {total_revised} segundos')

Optimization failed. The problem appears to be unbounded.
         Iterations: 4
     con: array([], dtype=float64)
     fun: -182.0
 message: 'Optimization failed. The problem appears to be unbounded.'
     nit: 4
   slack: array([5., 6., 0., 3.])
  status: 3
 success: False
       x: array([0., 7.])
Tiempo total con el método revised-simplex: 0.007999658584594727 segundos
